# VICTGOAL Bike Helmet - Reviews Analysis\n\n## Q2: GenAI Analysis of Customer Reviews\n\nThis notebook analyzes customer reviews for the VICTGOAL Bike Helmet using:\n- Sentence embeddings and clustering\n- Semantic search with FAISS\n- OpenAI API for feature extraction\n- Image generation based on review sentiment

## Part 1: Import Libraries and Setup

In [ ]:
!pip install sentence-transformers faiss-cpu scikit-learn openai python-dotenv -q

In [ ]:
import pandas as pd\nimport numpy as np\nimport json\nimport os\nfrom sentence_transformers import SentenceTransformer\nimport faiss\nfrom sklearn.cluster import KMeans\nfrom sklearn.manifold import TSNE\nfrom sklearn.decomposition import PCA\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom dotenv import load_dotenv\nfrom openai import OpenAI\n\n# Set up plotting style\nsns.set_style('whitegrid')\nplt.rcParams['figure.figsize'] = (12, 8)

## Part 2: Load and Preprocess Reviews Data

In [ ]:
# Load the reviews dataset\ncsv_path = 'reviews.csv'\ndf = pd.read_csv(csv_path)\n\nprint(f\"Loaded {len(df)} rows\")\ndf.head()

In [ ]:
# Extract reviews from 'tl-m' column (4th column, index 3)\ndf_reviews = df.iloc[:, [3]].copy()\n\n# Remove rows with NaN values\ndf_reviews = df_reviews.dropna()\n\n# Rename column to 'review'\ndf_reviews.columns = ['review']\n\n# Reset index\ndf_reviews = df_reviews.reset_index(drop=True)\n\nprint(f\"\\nCleaned reviews: {len(df_reviews)}\")\ndf_reviews.head()

In [ ]:
# Save cleaned reviews\noutput_path = 'cleaned_helmet_reviews.csv'\ndf_reviews.to_csv(output_path, index=False)\nprint(f\"Cleaned reviews saved to: {output_path}\")

## Part 3: Generate Embeddings and Create FAISS Index

In [ ]:
# Load sentence transformer model\nmodel = SentenceTransformer('all-MiniLM-L6-v2')\n\n# Generate embeddings for all reviews\nprint(\"Generating embeddings...\")\nembeddings = model.encode(df_reviews['review'].tolist(), show_progress_bar=True)\nprint(f\"Generated {len(embeddings)} embeddings with dimension {embeddings.shape[1]}\")

In [ ]:
# Convert to float32 and normalize for FAISS\nembeddings_np = np.array(embeddings, dtype='float32')\nfaiss.normalize_L2(embeddings_np)\n\n# Create FAISS index\nd = embeddings_np.shape[1]  # dimension\nindex = faiss.IndexFlatIP(d)  # Inner Product (cosine similarity after normalization)\nindex.add(embeddings_np)\n\nprint(f\"FAISS index created with {index.ntotal} vectors\")

## Part 4: K-means Clustering for Topic Analysis

In [ ]:
# Perform K-means clustering\nn_clusters = 8\nkmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)\ncluster_labels = kmeans.fit_predict(embeddings_np)\n\n# Add cluster labels to dataframe\ndf_reviews['cluster'] = cluster_labels\n\nprint(f\"Created {n_clusters} clusters\")\nprint(df_reviews['cluster'].value_counts().sort_index())

In [ ]:
# Visualize cluster sample reviews\nprint(\"\\nCLUSTER TOPIC ANALYSIS\\n\")\nfor cluster_id in range(n_clusters):\n    cluster_reviews = df_reviews[df_reviews['cluster'] == cluster_id]\n    sample_review = cluster_reviews['review'].iloc[0]\n    print(f\"Cluster {cluster_id} ({len(cluster_reviews)} reviews):\")\n    print(f\"  Sample: {sample_review[:150]}...\\n\")

## Part 5: Semantic Search Functions with FAISS

In [ ]:
def search_reviews(query, top_k=50):\n    \"\"\"Search reviews using FAISS semantic search\"\"\"\n    # Encode query\n    q_emb = model.encode([query], convert_to_numpy=True)\n    q_emb = q_emb.astype('float32')\n    faiss.normalize_L2(q_emb)\n    \n    # Search in FAISS\n    scores, indices = index.search(q_emb, top_k)\n    \n    indices = indices[0]\n    scores = scores[0]\n    \n    # Get results\n    results = df_reviews.iloc[indices].copy()\n    results['score'] = scores\n    return results

In [ ]:
# Test semantic search\ntest_query = \"magnetic visor and goggles design, light features, ventilation\"\nresults = search_reviews(test_query, top_k=10)\nprint(f\"Top 5 results for query: '{test_query}'\\n\")\nfor i, row in results.head().iterrows():\n    print(f\"Score: {row['score']:.3f}\")\n    print(f\"Review: {row['review'][:200]}...\\n\")

## Part 6: Retrieve Reviews for Visual Features Analysis

In [ ]:
# Define search queries for different aspects of the bike helmet\nvisual_query = \"\"\"\nReviews describing the bike helmet's visual appearance and features:\nmagnetic visor design, detachable goggles, USB rechargeable light, LED light positions,\nhelmet shape and form, ventilation holes pattern, color options, overall design style,\nadjustable dial system, material texture.\n\"\"\"\n\nfunction_query = \"\"\"\nDescriptions of how the bike helmet operates and functions:\nvisor attachment mechanism, goggles magnetic system, light brightness and modes,\nventilation airflow, fit adjustment, padding comfort, safety protection,\nease of use, weight and balance.\n\"\"\"\n\nmaterials_query = \"\"\"\nComments about materials, textures, and build quality:\nEPS foam structure, PC shell material, padding quality and comfort,\nvisor and goggle materials, light housing, overall durability and sturdiness,\nquality of construction.\n\"\"\"\n\npositive_query = \"\"\"\nReviews praising the helmet's design and features:\nattractive appearance, innovative visor/goggle system, effective LED light,\ngood ventilation design, comfortable fit, sleek modern look, quality build.\n\"\"\"\n\nnegative_query = \"\"\"\nComplaints about the helmet's appearance or features:\nvisor doesn't fit well, goggles fall off, light too dim or bright,\npoor ventilation, uncomfortable fit, looks cheap, sizing issues,\nbuild quality concerns.\n\"\"\"

In [ ]:
# Retrieve relevant reviews for each query\ntop_visual_reviews = search_reviews(visual_query, top_k=400)\ntop_function_reviews = search_reviews(function_query, top_k=400)\ntop_materials_reviews = search_reviews(materials_query, top_k=400)\ntop_positive_reviews = search_reviews(positive_query, top_k=400)\ntop_negative_reviews = search_reviews(negative_query, top_k=400)\n\nprint(f\"Retrieved visual-related reviews: {len(top_visual_reviews)}\")\nprint(f\"Retrieved functional-related reviews: {len(top_function_reviews)}\")\nprint(f\"Retrieved materials-related reviews: {len(top_materials_reviews)}\")\nprint(f\"Retrieved positive design reviews: {len(top_positive_reviews)}\")\nprint(f\"Retrieved negative design reviews: {len(top_negative_reviews)}\")

## Part 7: Sample Reviews for Analysis

In [ ]:
# Combine and deduplicate reviews\ncore_visual_df = pd.concat(\n    [top_visual_reviews, top_function_reviews, top_materials_reviews],\n    ignore_index=True\n)\n\nprint(f\"Combined reviews: {len(core_visual_df)}\")\n\n# Remove duplicates\ncore_visual_df = core_visual_df.drop_duplicates(subset=['review']).reset_index(drop=True)\nprint(f\"After removing duplicates: {len(core_visual_df)}\")\n\n# Sample if needed\nmax_core_reviews = 500\nif len(core_visual_df) > max_core_reviews:\n    core_visual_df = core_visual_df.sample(n=max_core_reviews, random_state=42).reset_index(drop=True)\n    print(f\"Sampled to: {len(core_visual_df)} reviews\")

In [ ]:
# Prepare text for OpenAI analysis\nall_core_reviews_text = \"\\n\\n---REVIEW---\\n\\n\".join(core_visual_df['review'].tolist())\n\nprint(f\"Total characters: {len(all_core_reviews_text)}\")\nprint(f\"Estimated tokens: {len(all_core_reviews_text) // 4}\")

## Part 8: OpenAI API Feature Extraction

In [ ]:
# Load OpenAI API key\nload_dotenv()\nOPENAI_API_KEY = os.environ.get(\"OPENAI_API_KEY\")\nclient = OpenAI(api_key=OPENAI_API_KEY)\nprint(\"OpenAI client initialized\")

In [ ]:
# Prepare analysis prompt\nanalysis_prompt = f\"\"\"You are an expert product analyst. Analyze the following {len(core_visual_df)} customer reviews for a bike helmet (VICTGOAL Bike Helmet with Visor and Goggles).\n\nYour task is to extract VISUAL and PHYSICAL information useful for generating product images. Focus on:\n\n1. **Overall Design**: Shape, style, modern/sporty aesthetic\n2. **Key Features**: Magnetic visor, detachable goggles, USB rechargeable LED light\n3. **Materials & Textures**: EPS foam, PC shell, padding materials\n4. **Design Elements**: Ventilation holes pattern (21 vents), color options, reflective elements\n5. **Components**: Adjustment dial, goggle attachment system, light position\n6. **Size & Proportions**: Compact, streamlined, lightweight appearance\n\nReturn a JSON object with this structure:\n{{\n  \"design\": {{\n    \"overall_style\": \"description\",\n    \"shape\": \"description\",\n    \"aesthetic\": \"description\"\n  }},\n  \"materials\": {{\n    \"shell\": \"description\",\n    \"foam\": \"description\",\n    \"padding\": \"description\",\n    \"quality\": \"description\"\n  }},\n  \"key_features\": {{\n    \"visor\": \"description of magnetic visor\",\n    \"goggles\": \"description of goggles attachment\",\n    \"light\": \"description of LED light system\",\n    \"ventilation\": \"description of vent pattern\"\n  }},\n  \"colors\": {{\n    \"available\": [\"list of colors mentioned\"],\n    \"accents\": [\"reflective elements, accents\"]\n  }},\n  \"keywords\": [\"list of visual descriptive keywords\"],\n  \"image_prompt\": \"A single detailed paragraph describing the bike helmet visually for image generation\"\n}}\n\nReviews:\n{all_core_reviews_text}\n\"\"\"\n\nprint(\"Sending request to OpenAI API...\")

In [ ]:
# Call OpenAI API\ntry:\n    response = client.chat.completions.create(\n        model=\"gpt-4o\",\n        messages=[\n            {\"role\": \"system\", \"content\": \"You are an expert product analyst and visual designer.\"},\n            {\"role\": \"user\", \"content\": analysis_prompt}\n        ],\n        response_format={\"type\": \"json_object\"},\n        temperature=0.7\n    )\n    \n    print(\"Response received!\\n\")\n    \n    # Extract and parse response\n    response_text = response.choices[0].message.content\n    features_json = json.loads(response_text)\n    \n    print(\"EXTRACTED VISUAL FEATURES\")\n    print(json.dumps(features_json, indent=2))\n    \n    # Save to file\n    with open('helmet_extracted_features.json', 'w') as f:\n        json.dump(features_json, f, indent=2)\n    print(\"\\nFeatures saved to: helmet_extracted_features.json\")\n    \n    # Extract image generation prompt\n    image_prompt = features_json.get('image_prompt', '')\n    print(\"\\nFINAL IMAGE GENERATION PROMPT\")\n    print(image_prompt)\n    \nexcept Exception as e:\n    print(f\"Error calling OpenAI API: {e}\")

## Part 9: Image Generation with DALL-E

In [ ]:
# Generate image using DALL-E\ntry:\n    image_response = client.images.generate(\n        model=\"dall-e-3\",\n        prompt=image_prompt,\n        size=\"1024x1024\",\n        quality=\"standard\",\n        n=1\n    )\n    \n    image_url = image_response.data[0].url\n    print(\"Image generated successfully!\")\n    print(f\"Image URL: {image_url}\")\n    \n    # Save image URL\n    with open('generated_helmet_image.txt', 'w') as f:\n        f.write(image_url)\n    \nexcept Exception as e:\n    print(f\"Error generating image: {e}\")

## Part 10: Summary and Conclusions

In [ ]:
print(\"\\n=== ANALYSIS SUMMARY ===\\n\")\nprint(f\"Total reviews analyzed: {len(df_reviews)}\")\nprint(f\"Number of clusters identified: {n_clusters}\")\nprint(f\"Reviews used for visual analysis: {len(core_visual_df)}\")\nprint(\"\\nKey findings extracted and saved to helmet_extracted_features.json\")\nprint(\"Product image generated based on customer review insights\")